<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
# Pandas per la gestione dei dati
import pandas as pd
import requests

from collections import Counter
from collections import defaultdict

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [219]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2020s.csv"
response = requests.get(url)

with open("charting-m-points-2020s.csv", "w") as f:
    f.write(response.text)

dataset1 = pd.read_csv("charting-m-points-2020s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2010s.csv"
response = requests.get(url)

with open("charting-m-points-2010s.csv", "w") as f:
    f.write(response.text)

dataset2 = pd.read_csv("charting-m-points-2010s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-to-2009.csv"
response = requests.get(url)

with open("charting-m-points-to-2009.csv", "w") as f:
    f.write(response.text)

dataset3 = pd.read_csv("charting-m-points-to-2009.csv")


dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)

dataset

<ipython-input-219-ee841c57e572>:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset1 = pd.read_csv("charting-m-points-2020s.csv")


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0.0,0.0,0-0,1.0,True,1,6s17y3w@,NaN,192,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0.0,0.0,0-15,1.0,True,1,6w,6f29f2f2f3b3f2f3r2d#,159,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0.0,0.0,0-30,1.0,True,1,4#,NaN,189,1
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0.0,0.0,15-30,1.0,True,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,158,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0.0,0.0,15-40,1.0,True,1,6n,5f28f3b1s3f1d@,156,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1


Numero di partite del dataset

In [220]:
simboli_finali = dataset['match_id'].nunique()
simboli_finali

6005

Analisi delle tipologie degli esiti finali dei punti, analisi dei casi rari

In [221]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]
    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 339684
'*': 323597
'@': 312908
'n': 15391
'd': 15066
'S': 7548
'w': 4055
'R': 2242
'x': 892
'C': 304
'g': 97
'Q': 60
'!': 49
' ': 39
'P': 37
'e': 32
'3': 26
'1': 22
'9': 11
'8': 8
'2': 7
'+': 7
'7': 4
'=': 3
'^': 2
'b': 2
'c': 2
'4': 1
'&': 1
')': 1
'v': 1
'p': 1
'.': 1
'-': 1
'm': 1
'z': 1


Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile.

La descrizione in formato semplice, una riga per ogni stringa:


- `1` = `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `2` = `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `3` = `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `4` = `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `5` = `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `6` = `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `7` = `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `8` = `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `9` = `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `A` = `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `B` = `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `C` = `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

Attualmente non ho aggiunto considerazioni sul primo e secondo servizio, e su posizionamenti della palla e tipologia di colpi giocati.

Ho considerato le challenges Hawk-Eye perse come errori forzati.
Errori forzati e winners sono stati considerati come la stessa cosa nei servizi, si potrebbe fare la stessa considerazione anche nei rally

In [222]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,PointType
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0.0,0.0,0-0,1.0,True,1,6s17y3w@,NaN,192,2,C
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0.0,0.0,0-15,1.0,True,1,6w,6f29f2f2f3b3f2f3r2d#,159,2,A
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0.0,0.0,0-30,1.0,True,1,4#,NaN,189,1,3
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0.0,0.0,15-30,1.0,True,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,158,2,C
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0.0,0.0,15-40,1.0,True,1,6n,5f28f3b1s3f1d@,156,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1,1


In [223]:
simboli_finali = dataset['PointType'].value_counts()

simboli_finali

,count
PointType,
C,156269
B,153760
9,150697
A,146649
7,120676
8,118525
3,65918
4,61120
6,18986


Creazione delle progressioni punteggio

In [224]:
new_data = []
current_id = None
current_set = None
current_game = None
last_point = None
new_row = None

for index, row in dataset.iterrows():
    match_id = row['match_id']
    set_score = row['Set1'] + row['Set2']
    game_score = row['Gm1'] + row['Gm2']
    point_winner = row['PtWinner']
    point_type = row['PointType']

    # If New match
    if match_id != current_id:
        # Save previous Match labels
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point
              if r['MatchWinner'] == 0:
                  r['MatchWinner'] = last_point


        current_id = match_id
        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'PoweredGmProgression': [point_type],
            "Svr": row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If New set of the same Match
    elif set_score != current_set:
        # Save previous Set labels
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point



        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'PoweredGmProgression': [point_type],
            'Svr': row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If New game of the same Set
    elif game_score != current_game:
        # Save previous Game label
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)


        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'PoweredGmProgression': [point_type],
            'Svr': row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If same Game, Set and Match of the previous consideration
    else:
        # Save Game progression sequences
        if new_row:
            new_row['GmProgression'].append(point_winner)
            new_row['PoweredGmProgression'].append(point_type)

    last_point = point_winner

# Last dataset's game memorization
if new_row and last_point:
    new_row['GmWinner'] = last_point
    new_data.append(new_row)
    for r in new_data:
      if r['SetWinner'] == 0:
          r['SetWinner'] = last_point
      if r['MatchWinner'] == 0:
          r['MatchWinner'] = last_point




new_dataset = pd.DataFrame(new_data)
new_dataset

,match_id,GmProgression,PoweredGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 1, 2, 1, 2]","[C, A, 3, C, B, A]",1,0,0,0.0,0,2,2,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 2, 1, 1, 2]","[8, 4, 4, 5, 5, 4]",2,0,0,0.0,1,2,2,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[1, 1, 1, 1]","[3, 3, B, 7]",1,0,0,0.0,2,1,2,2
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 1, 2, 1, 2]","[8, 4, 9, C, B, 4]",2,0,0,1.0,2,2,2,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 1, 2, 1, 2, 1, 1, 1]","[C, 7, C, 3, A, B, 3, 3]",1,0,0,1.0,3,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...
165045,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[1, 1, 1, 1]","[1, 1, 1, 1]",1,2,2,2.0,2,1,1,1
165046,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[2, 2, 2, 2]","[2, 2, 2, 2]",2,2,2,3.0,2,2,1,1
165047,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[1, 1, 1, 1]","[1, 1, 1, 1]",1,2,2,3.0,3,1,1,1
165048,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[1, 1, 1, 1]","[1, 1, 1, 1]",2,2,2,4.0,3,1,1,1


Analisi Bilanciamento Etichette

In [225]:
simboli_finali = new_dataset['GmWinner'].value_counts()

simboli_finali

,count
GmWinner,
1,83948
2,81102


In [226]:
simboli_finali = new_dataset['SetWinner'].value_counts()

simboli_finali

,count
SetWinner,
1,84626
2,80424


In [227]:
simboli_finali = new_dataset['MatchWinner'].value_counts()

simboli_finali

,count
MatchWinner,
1,85220
2,79830


Prendo la lista dei giocatori di Tennis

In [262]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv"
response = requests.get(url)

with open("atp_players.csv", "w") as f:
    f.write(response.text)

playerset = pd.read_csv("atp_players.csv")

# Sostituisci NaN con stringhe vuote prima della concatenazione
first = playerset['name_first'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)
last = playerset['name_last'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)

# Costruisci il full_name in modo sicuro
playerset['full_name'] = (
    first + '_' + last
).str.strip('_')  # Rimuove eventuali underscore iniziali/finali se una parte è vuota
playerset['full_name'] = playerset['full_name'].str.lower()
playerset

<ipython-input-262-48ffade97733>:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  playerset = pd.read_csv("atp_players.csv")


,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,full_name
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544,gardnar_mulloy
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581,pancho_segura
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049,frank_sedgman
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752,giuseppe_merlo
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554,richard_gonzalez
...,...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN,matvei_kobiakov
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN,tobia_costanzo_baragiola_mordini
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN,dominik_wijntjes
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN,sam_wensley


Prendo i rankings degli ultimi 20 anni

In [229]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_current.csv"
response = requests.get(url)

with open("atp_rankings_current.csv", "w") as f:
    f.write(response.text)

ranking1 = pd.read_csv("atp_rankings_current.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_20s.csv"
response = requests.get(url)

with open("atp_rankings_20s.csv", "w") as f:
    f.write(response.text)

ranking2 = pd.read_csv("atp_rankings_20s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_10s.csv"
response = requests.get(url)

with open("atp_rankings_10s.csv", "w") as f:
    f.write(response.text)

ranking3 = pd.read_csv("atp_rankings_10s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_00s.csv"
response = requests.get(url)

with open("atp_rankings_00s.csv", "w") as f:
    f.write(response.text)

ranking4 = pd.read_csv("atp_rankings_00s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_90s.csv"
response = requests.get(url)

with open("atp_rankings_90s.csv", "w") as f:
    f.write(response.text)

ranking5 = pd.read_csv("atp_rankings_90s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_80s.csv"
response = requests.get(url)

with open("atp_rankings_80s.csv", "w") as f:
    f.write(response.text)

ranking6 = pd.read_csv("atp_rankings_80s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_70s.csv"
response = requests.get(url)

with open("atp_rankings_70s.csv", "w") as f:
    f.write(response.text)

ranking7 = pd.read_csv("atp_rankings_70s.csv")

rankingset = pd.concat([ranking1, ranking2, ranking3, ranking4, ranking5, ranking6, ranking7], ignore_index=True)

rankingset

,ranking_date,rank,player,points
0,20240101,1,104925,11245.0
1,20240101,2,207989,8855.0
2,20240101,3,106421,7600.0
3,20240101,4,206173,6490.0
4,20240101,5,126094,4805.0
...,...,...,...,...
3292944,19771212,95,100586,NaN
3292945,19771212,97,100363,NaN
3292946,19771212,98,100176,NaN
3292947,19771212,99,100233,NaN


Ottimizzazione e Correzione causa errori dataset

In [292]:
# Crea il dizionario: full_name -> lista di player_id per ottimizzare l'esecuzione
player_id_map = defaultdict(list)

for _, row in playerset.iterrows():
  player_id_map[row['full_name']].append(row['player_id'])
  if(row['full_name']=='franko_skugor'):
    player_id_map['franco_skugor'].append(row['player_id'])
  if(row['full_name']=='jo_wilfried_tsonga'):
    player_id_map['jo_wilfred_tsonga'].append(row['player_id'])
  if(row['full_name']=='sam_groth'):
    player_id_map['samuel_groth'].append(row['player_id'])
  if(row['full_name']=='andres_artunedo_martinavarro'):
    player_id_map['andres_artunedo'].append(row['player_id'])
  if(row['full_name']=='marc_kevin_goellner'):
    player_id_map['mark_kevin_goellner'].append(row['player_id'])



Dataset con Ranking Giocatori

In [293]:
last_match = None
ranking_1 = None
ranking_2 = None

def extract_rankings(row):
    global last_match, ranking_1, ranking_2
    match_id = row['match_id']
    if match_id == last_match:
      return pd.Series([ranking_1, ranking_2])
    else:
      last_match = match_id

      match_info = match_id.split('-')
      ranking_date = match_info[0]
      name1 = match_info[-2].lower()
      name2 = match_info[-1].lower()

      # Trova i player_id in modo sicuro
      player_id1 = player_id_map.get(name1)
      player_id2 = player_id_map.get(name2)

      # Converte la data di riferimento 'ranking_date' in formato datetime
      ranking_date = pd.to_datetime(ranking_date, format='%Y%m%d')

      print(match_id)

      for player_id in player_id1:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= ranking_date)
              ]
              if not filtered_ranking.empty:
                  ranking_1 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  break  # Esce dal ciclo appena trova un ranking valido
          except Exception as e:
              continue  # Salta in caso di errore e prova con il prossimo ID

      for player_id in player_id2:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= ranking_date)
              ]
              if not filtered_ranking.empty:
                  ranking_2 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  break
          except Exception as e:
              continue

      return pd.Series([ranking_1, ranking_2])


new_dataset[['Ranking1', 'Ranking2']] = new_dataset.apply(extract_rankings, axis=1)
new_dataset

Output streaming troncato alle ultime 5000 righe.
20230520-M-Rome_Masters-SF-Stefanos_Tsitsipas-Daniil_Medvedev
20230520-M-Rome_Masters-SF-Casper_Ruud-Holger_Rune
20230518-M-Rome_Masters-QF-Yannick_Hanfmann-Daniil_Medvedev
20230516-M-Rome_Masters-R16-Francisco_Cerundolo-Jannik_Sinner
20230516-M-Rome_Masters-R16-Casper_Ruud-Laslo_Djere
20230516-M-Rome_Masters-R16-Alexander_Zverev-Daniil_Medvedev
20230515-M-Rome_Masters-R32-Fabian_Marozsan-Carlos_Alcaraz
20230515-M-Rome_Masters-R32-Bernabe_Zapata_Miralles-Daniil_Medvedev
20230515-M-Rome_Masters-R32-Alejandro_Davidovich_Fokina-Andrey_Rublev
20230514-M-Rome_Masters-R64-Taylor_Fritz-Yannick_Hanfmann
20230514-M-Rome_Masters-R64-Stefanos_Tsitsipas-Nuno_Borges
20230514-M-Rome_Masters-R64-Emil_Ruusuvuori-Daniil_Medvedev
20230514-M-Rome_Masters-R32-Fabio_Fognini-Holger_Rune
20230514-M-Rome_Masters-R32-Alexander_Shevchenko-Jannik_Sinner
20230513-M-Rome_Masters-R64-J_J_Wolf-Hubert_Hurkacz
20230513-M-Rome_Masters-R64-Carlos_Alcaraz-Albert_Ramos
202

,match_id,GmProgression,PoweredGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner,Ranking1,Ranking2
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 1, 2, 1, 2]","[C, A, 3, C, B, A]",1,0,0,0.0,0,2,2,2,109,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 2, 1, 1, 2]","[8, 4, 4, 5, 5, 4]",2,0,0,0.0,1,2,2,2,109,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[1, 1, 1, 1]","[3, 3, B, 7]",1,0,0,0.0,2,1,2,2,109,2
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 1, 2, 1, 2]","[8, 4, 9, C, B, 4]",2,0,0,1.0,2,2,2,2,109,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 1, 2, 1, 2, 1, 1, 1]","[C, 7, C, 3, A, B, 3, 3]",1,0,0,1.0,3,1,2,2,109,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165045,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[1, 1, 1, 1]","[1, 1, 1, 1]",1,2,2,2.0,2,1,1,1,11,7
165046,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[2, 2, 2, 2]","[2, 2, 2, 2]",2,2,2,3.0,2,2,1,1,11,7
165047,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[1, 1, 1, 1]","[1, 1, 1, 1]",1,2,2,3.0,3,1,1,1,11,7
165048,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,"[1, 1, 1, 1]","[1, 1, 1, 1]",2,2,2,4.0,3,1,1,1,11,7


In [294]:
new_dataset.to_csv('new_dataset.csv', index=False)